# Загружаем изображение в LabelStudio, делаем предсказание и загружаем предсказания

Вроде как изображения в папке должны быть названы в соответствии с предполагаемым inner_id

Настроить конфигурацию (api)

In [ ]:
import os
import requests
import uuid
from ultralytics import YOLO

# Конфигурация
LS_URL = ''
API_KEY = ''
PROJECT_ID = 000
START_INNER_ID = 339 # Последний Inner_id в проекте LS
IMAGE_DIR = r"D:\НЕ ШКИЛА\СТАЖЕРКА\Изображения\5_2\ALL"
MODEL_PATH = r"D:\НЕ ШКИЛА\СТАЖЕРКА\222\custom_yolo_model.pt"
TOTAL_IMAGES = 702  # Общее количество изображений для загрузки

# Загрузка модели
model = YOLO(MODEL_PATH)

# Генерируем список файлов
image_files = [f"{i}.jpg" for i in range(1, TOTAL_IMAGES + 1)]

# Загрузка изображений в проект
for img_file in image_files:
    img_path = os.path.join(IMAGE_DIR, img_file)

    if not os.path.exists(img_path):
        print(f"⚠️ Файл не найден: {img_path}")
        continue

    with open(img_path, 'rb') as f:
        response = requests.post(
            f"{LS_URL}/api/projects/{PROJECT_ID}/import",
            headers={"Authorization": f"Token {API_KEY}"},
            files=[('file', (img_file, f, 'image/jpeg'))]
        )

    if response.status_code == 201:
        print(f"✅ Загружено: {img_file}")
    else:
        print(f"❌ Ошибка загрузки {img_file}: {response.status_code} {response.text}")

# Получение списка задач
response = requests.get(
    f"{LS_URL}/api/tasks",
    headers={"Authorization": f"Token {API_KEY}"},
    params={"project": PROJECT_ID, "page_size": 1500}
)

if response.status_code != 200:
    print(f"❌ Ошибка получения задач: {response.status_code} {response.text}")
    exit()

all_tasks = response.json().get("tasks", [])
print(f"Получено {len(all_tasks)} задач")

# Фильтрация задач по диапазону inner_id
target_tasks = []
for inner_id in range(START_INNER_ID, START_INNER_ID + TOTAL_IMAGES):
    task = next((t for t in all_tasks if t['inner_id'] == inner_id), None)
    if task:
        target_tasks.append(task)
        print(f"🔍 Найдена задача: inner_id={inner_id}, task_id={task['id']}")
    else:
        print(f"⚠️ Задача с inner_id={inner_id} не найдена!")

if not target_tasks:
    print("❌ Не найдено ни одной задачи в указанном диапазоне!")
    exit()

print(f"\nНайдено задач в диапазоне {START_INNER_ID}-{START_INNER_ID + TOTAL_IMAGES - 1}: {len(target_tasks)}")

# Обработка и загрузка предсказаний
for task in target_tasks:
    img_number = task['inner_id'] - START_INNER_ID + 1
    img_file = f"{img_number}.jpg"
    img_path = os.path.join(IMAGE_DIR, img_file)

    if not os.path.exists(img_path):
        print(f"⚠️ Файл не найден: {img_path}")
        continue

    # Делаем предсказание
    results = model.predict(img_path, conf=0.5)
    annotations = []

    for box in results[0].boxes:
        x1, y1, x2, y2 = box.xyxyn[0].tolist()
        width = (x2 - x1) * 100
        height = (y2 - y1) * 100

        annotations.append({
            "id": str(uuid.uuid4()),
            "type": "rectanglelabels",
            "value": {
                "x": x1 * 100,
                "y": y1 * 100,
                "width": width,
                "height": height,
                "rotation": 0,
                "rectanglelabels": [model.names[int(box.cls)]]
            },
            "to_name": "image",
            "from_name": "label",
            "score": float(box.conf)
        })

    # Формируем предсказание
    prediction = {
        "task": task["id"],
        "result": annotations,
        "model_version": "yolo_model"
    }

    # Отправляем предсказание
    response = requests.post(
        f"{LS_URL}/api/predictions",
        headers={"Authorization": f"Token {API_KEY}", "Content-Type": "application/json"},
        json=prediction
    )

    if response.status_code == 201:
        print(f"✅ Успешно: задача {task['id']} (inner_id: {task['inner_id']}) - {img_file}")
    else:
        print(f"❌ Ошибка для задачи {task['id']}: {response.status_code} {response.text}")

print("\n" + "=" * 50)
print(f"Все операции завершены! Обработано {len(target_tasks)} задач")
print("=" * 50)